In [1]:
from binance.client import Client
import datetime
import pandas as pd
import os

api_key = "emptyKey"
api_secret = "emptySecret"
client = Client(api_key, api_secret)

def listToDataframe(dataUnprocess):
    data_df = pd.DataFrame(dataUnprocess)
    data_df = data_df.drop(columns=[i for i in range(6, 12)])
    data_df.columns = ['TimeStamp', 'Open', 'High', 'Low', 'Close', 'Volume']
    data_df['Datetime'] = data_df['TimeStamp'].apply(lambda x: datetime.datetime.fromtimestamp(x//1000).strftime("%Y-%m-%d %H:%M:%S"))
    data_df['Datetime'] = pd.to_datetime(data_df['Datetime']) - datetime.timedelta(hours=8) #轉為UTC+0的時區
    data_df['Date'] = data_df['Datetime'].apply(lambda x: str(x)[:10])
    data_df['Time'] = data_df['Datetime'].apply(lambda x: str(x)[11:])
    data_df['Time'] = data_df['Time'].apply(lambda x: str(x)[:5])
    data_df = data_df.set_index('Datetime')
    data_df = data_df.drop(columns=['TimeStamp', 'Date', 'Time'])
    data_df = data_df[['Open', 'High', 'Low', 'Close', 'Volume']]
    return data_df    
    
def getData(symbol="LTCBTC", interval=Client.KLINE_INTERVAL_1HOUR, startTime="1 day ago UTC"):
    all_df = pd.DataFrame()
    nowTime = datetime.date.today()
    while True:
        endTime = pd.to_datetime(startTime)+datetime.timedelta(days=1)
        endTime = endTime.strftime("%Y-%m-%d")
        # fetch klines
        klines = client.get_historical_klines(symbol, interval, startTime, endTime)    
        if klines:
            temp_data_df = listToDataframe(klines)
            all_df = all_df.append(temp_data_df)
       
        if pd.to_datetime(startTime) < pd.to_datetime(nowTime):
            startTime = endTime
        else:
            break    
    all_df = all_df.sort_index()
    all_df = all_df.drop_duplicates()
    all_df = all_df.dropna()
    
    return all_df

In [2]:
tickers = client.get_all_tickers()
spot = []

for i in range(len(tickers)):
    if tickers[i]['symbol'][-4:] == 'USDT':
        spot.append(tickers[i]['symbol'][:-4])

In [3]:
spot

['BTC',
 'ETH',
 'BNB',
 'BCC',
 'NEO',
 'LTC',
 'QTUM',
 'ADA',
 'XRP',
 'EOS',
 'TUSD',
 'IOTA',
 'XLM',
 'ONT',
 'TRX',
 'ETC',
 'ICX',
 'VEN',
 'NULS',
 'VET',
 'PAX',
 'BCHABC',
 'BCHSV',
 'USDC',
 'LINK',
 'WAVES',
 'BTT',
 'USDS',
 'ONG',
 'HOT',
 'ZIL',
 'ZRX',
 'FET',
 'BAT',
 'XMR',
 'ZEC',
 'IOST',
 'CELR',
 'DASH',
 'NANO',
 'OMG',
 'THETA',
 'ENJ',
 'MITH',
 'MATIC',
 'ATOM',
 'TFUEL',
 'ONE',
 'FTM',
 'ALGO',
 'USDSB',
 'GTO',
 'ERD',
 'DOGE',
 'DUSK',
 'ANKR',
 'WIN',
 'COS',
 'NPXS',
 'COCOS',
 'MTL',
 'TOMO',
 'PERL',
 'DENT',
 'MFT',
 'KEY',
 'STORM',
 'DOCK',
 'WAN',
 'FUN',
 'CVC',
 'CHZ',
 'BAND',
 'BUSD',
 'BEAM',
 'XTZ',
 'REN',
 'RVN',
 'HC',
 'HBAR',
 'NKN',
 'STX',
 'KAVA',
 'ARPA',
 'IOTX',
 'RLC',
 'MCO',
 'CTXC',
 'BCH',
 'TROY',
 'VITE',
 'FTT',
 'EUR',
 'OGN',
 'DREP',
 'BULL',
 'BEAR',
 'ETHBULL',
 'ETHBEAR',
 'TCT',
 'WRX',
 'BTS',
 'LSK',
 'BNT',
 'LTO',
 'EOSBULL',
 'EOSBEAR',
 'XRPBULL',
 'XRPBEAR',
 'STRAT',
 'AION',
 'MBL',
 'COTI',
 'BNBBULL',
 'B

In [4]:
len(spot)

336

In [5]:
df = pd.DataFrame()

for ticker in spot:
    
    ticker = ticker+'USDT'
    test = client.get_historical_klines(ticker, interval = '1d', start_str = '2019-08-31 0:0:0', end_str = '2019-09-01 0:0:0')
    
    if len(test) != 0:
        try:
            each = getData(symbol = ticker, startTime = '2019-09-01 0:0:0')
        except:
            continue
        each = each[['Close']]
        each.columns = [ticker]
        each = each.resample(rule = '1H').last().fillna(method = "ffill")
        df = pd.concat([df, each], axis = 1)

In [10]:
df = df.iloc[:-5]

In [12]:
df = df.dropna(axis = 1)

In [15]:
df

,BTCUSDT,ETHUSDT,BNBUSDT,NEOUSDT,LTCUSDT,ADAUSDT,XRPUSDT,EOSUSDT,TUSDUSDT,IOTAUSDT,...,MTLUSDT,TOMOUSDT,PERLUSDT,DENTUSDT,MFTUSDT,KEYUSDT,DOCKUSDT,WANUSDT,FUNUSDT,CVCUSDT
Datetime,,,,,,,,,,,,,,,,,,,,,
2019-09-01 00:00:00,9617.06000000,172.45000000,21.35790000,8.82800000,64.70000000,0.04495000,0.25837000,3.32350000,0.99950000,0.24770000,...,0.40650000,0.47150000,0.08690000,0.00046060,0.00104200,0.00142500,0.00641200,0.34220000,0.00269800,0.04071000
2019-09-01 01:00:00,9614.99000000,172.71000000,21.45530000,8.83000000,64.66000000,0.04504000,0.25754000,3.32590000,0.99880000,0.24780000,...,0.40500000,0.47560000,0.08690000,0.00046120,0.00104700,0.00142500,0.00640000,0.33810000,0.00272300,0.04089000
2019-09-01 02:00:00,9605.78000000,172.43000000,21.31140000,8.82000000,64.60000000,0.04487000,0.25697000,3.31300000,0.99910000,0.24680000,...,0.40470000,0.47240000,0.08650000,0.00046040,0.00104600,0.00142500,0.00640000,0.33600000,0.00272600,0.04048000
2019-09-01 03:00:00,9623.02000000,172.57000000,21.43230000,8.84900000,64.81000000,0.04503000,0.25772000,3.31760000,0.99930000,0.24780000,...,0.40450000,0.47150000,0.08620000,0.00045980,0.00104000,0.00142500,0.00637500,0.33920000,0.00269300,0.04048000
2019-09-01 04:00:00,9614.46000000,172.46000000,21.53110000,8.83300000,64.82000000,0.04515000,0.25713000,3.32480000,0.99910000,0.24740000,...,0.40400000,0.47520000,0.08610000,0.00046000,0.00103900,0.00142500,0.00632900,0.33870000,0.00270000,0.04019000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-30 06:00:00,43371.92000000,3021.40000000,376.80000000,38.46000000,151.50000000,2.09300000,0.94410000,3.88800000,1.00000000,1.08920000,...,2.83300000,2.09100000,0.07795000,0.00447000,0.00958000,0.00728100,0.08113000,0.74950000,0.01858000,0.43520000
2021-09-30 07:00:00,43286.88000000,3001.59000000,375.10000000,38.11000000,150.70000000,2.08500000,0.94160000,3.87500000,1.00000000,1.08580000,...,2.81200000,2.10800000,0.07802000,0.00443500,0.00961500,0.00723800,0.08058000,0.74410000,0.01853000,0.43620000
2021-09-30 08:00:00,43062.03000000,2976.56000000,373.50000000,37.88000000,150.00000000,2.07200000,0.93390000,3.85600000,1.00000000,1.07430000,...,2.80800000,2.07100000,0.07692000,0.00442800,0.00951300,0.00717000,0.08035000,0.74090000,0.01851000,0.43300000


In [16]:
df.to_csv(os.getcwd() + '/crypto_data.csv')
